# Toxic Tweets Machine Learning Pipeline


## Dataset Preparation


In [1]:
import pandas as pd

RAW_DATASET_PATH = "FinalBalancedDataset.csv"

df = pd.read_csv(RAW_DATASET_PATH)
df = df[["Toxicity", "tweet"]]
df = df.rename(columns={"Toxicity": "is_toxic"})
df = df.sample(frac=1, ignore_index=True)

df.head()

,is_toxic,tweet
0,0,Three little birds
1,0,i like this positive #quote! never stress to t...
2,1,"RT @SoulYodeler: Go ahead and mock me, one day..."
3,0,wh is enemy deployed google for illegal manip....
4,1,I'm happy I ain't that bitch I use to be


In [2]:
import os

DATA_ROOT_PATH = "data"
if not os.path.exists(DATA_ROOT_PATH):
    os.makedirs(DATA_ROOT_PATH)

df.to_csv(os.path.join(DATA_ROOT_PATH, "data.csv"), index=False)

## Pipeline Development


In [3]:
import tensorflow as tf
import tensorflow_model_analysis as tfma
from tfx.components import (
    CsvExampleGen,
    StatisticsGen,
    SchemaGen,
    ExampleValidator,
    Transform,
    Trainer,
    Tuner,
    Evaluator,
)
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import (
    LatestBlessedModelStrategy,
)
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing
from tfx.proto import example_gen_pb2, trainer_pb2
from tfx.orchestration.experimental.interactive.interactive_context import (
    InteractiveContext,
)
from pprint import PrettyPrinter

2024-05-02 13:11:23.598382: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-02 13:11:23.642074: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-02 13:11:23.642113: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-02 13:11:23.643792: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-02 13:11:23.649844: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-02 13:11:23.650667: I tensorflow/core/platform/cpu_feature_guard.cc:1

### Set Variables


In [4]:
PIPELINE_NAME = "toxic-tweets-pipeline"
SCHEMA_PIPELINE_NAME = "toxic-tweets-tfdv-schema"

PIPELINE_ROOT = os.path.join("ramaastra-pipeline", PIPELINE_NAME)
METADATA_PATH = os.path.join("metadata", PIPELINE_NAME, "metadata.db")

SERVING_MODEL_DIR = os.path.join("serving_model", PIPELINE_NAME)

In [5]:
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

### Data Ingeston


#### Creating ExampleGen Component


In [6]:
output = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(
        splits=[
            example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
            example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2),
        ]
    )
)

In [7]:
example_gen = CsvExampleGen(input_base=DATA_ROOT_PATH, output_config=output)
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

#### Viewing the Raw Data


In [8]:
pp = PrettyPrinter()

train_uri = os.path.join(example_gen.outputs["examples"].get()[0].uri, "Split-train")

tfrecord_train_filenames = [
    os.path.join(train_uri, name) for name in os.listdir(train_uri)
]

dataset = tf.data.TFRecordDataset(tfrecord_train_filenames, compression_type="GZIP")

for tfrecord in dataset.take(5):
    serialized_example = tfrecord.numpy()
    example = tf.train.Example()
    example.ParseFromString(serialized_example)
    pp.pprint(example)

features {
  feature {
    key: "is_toxic"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "tweet"
    value {
      bytes_list {
        value: "Three little birds"
      }
    }
  }
}

features {
  feature {
    key: "is_toxic"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "tweet"
    value {
      bytes_list {
        value: "i like this positive #quote! never stress to things we can\'t control.     #life "
      }
    }
  }
}

features {
  feature {
    key: "is_toxic"
    value {
      int64_list {
        value: 1
      }
    }
  }
  feature {
    key: "tweet"
    value {
      bytes_list {
        value: "RT @SoulYodeler: Go ahead and mock me, one day I\'ll sell enough of this make-up to drive the pink Mary Kay Cadillac and we\'ll see who\'s gay&#8230;"
      }
    }
  }
}

features {
  feature {
    key: "is_toxic"
    value {
      int64_list {
        value: 1
      }
    }
  }
  feature {
    key

### Data Validation


#### Creating StatisticsGen Component


In [9]:
statistics_gen = StatisticsGen(examples=example_gen.outputs["examples"])
interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

#### Viewing the Data Statistics


In [10]:
interactive_context.show(statistics_gen.outputs["statistics"])

#### Creating SchemaGen Component


In [11]:
schema_gen = SchemaGen(statistics=statistics_gen.outputs["statistics"])
interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

#### Viewing Data Schema


In [12]:
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'is_toxic',INT,required,,-
'tweet',BYTES,required,,-


#### Creating the ExampleValidator Component


In [13]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs["statistics"], schema=schema_gen.outputs["schema"]
)
interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

#### Checking Anomalies from the Data


In [14]:
interactive_context.show(example_validator.outputs["anomalies"])

### Data Preprocessing/Transform


#### Creating the Transform Module


In [15]:
TRANSFORM_MODULE_FILE = "toxic_tweets_transform.py"

In [17]:
%%writefile {TRANSFORM_MODULE_FILE}

import tensorflow as tf

LABEL_KEY = "is_toxic"
FEATURE_KEY = "tweet"


def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"


regex_patterns = {
    "username": r"@[a-zA-z0-9_]+",
    "retweet": r"rt\s",
    "ampersand": r"&amp",
    "hashtag": r"#[a-zA-Z_\-\d]+",
    "url": r"https?:\/\/\S+|www\.\S+",
    "nonalphanum": r"[^a-zA-Z\d\s+]",
    "numbers": r"\d",
    "multispaces": r"\s+",
}


def preprocessing_fn(inputs):
    """
    Preprocess input features into transformed features
    Args:
        inputs: map from feature keys to raw features.
    Return:
        outputs: map from feature keys to transformed features.
    """

    outputs = {}

    preprocessed_text = tf.strings.lower(inputs[FEATURE_KEY])

    # Remove usernames, rt placeholders, ampersand, hashtags, urls, and unnecessary characters
    preprocessed_text = tf.strings.regex_replace(
        preprocessed_text, regex_patterns["username"], " "
    )
    preprocessed_text = tf.strings.regex_replace(
        preprocessed_text, regex_patterns["retweet"], " "
    )
    preprocessed_text = tf.strings.regex_replace(
        preprocessed_text, regex_patterns["ampersand"], " "
    )
    preprocessed_text = tf.strings.regex_replace(
        preprocessed_text, regex_patterns["hashtag"], " "
    )
    preprocessed_text = tf.strings.regex_replace(
        preprocessed_text, regex_patterns["url"], " "
    )
    preprocessed_text = tf.strings.regex_replace(
        preprocessed_text, regex_patterns["nonalphanum"], " "
    )
    preprocessed_text = tf.strings.regex_replace(
        preprocessed_text, regex_patterns["numbers"], " "
    )
    preprocessed_text = tf.strings.regex_replace(
        preprocessed_text, regex_patterns["multispaces"], " "
    )
    preprocessed_text = tf.strings.strip(preprocessed_text)

    outputs[transformed_name(FEATURE_KEY)] = preprocessed_text
    outputs[transformed_name(LABEL_KEY)] = tf.cast(inputs[LABEL_KEY], tf.int64)

    return outputs

Writing toxic_tweets_transform.py


#### Creating the Transform Component


In [18]:
transform = Transform(
    examples=example_gen.outputs["examples"],
    schema=schema_gen.outputs["schema"],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE),
)

interactive_context.run(transform)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying toxic_tweets_transform.py -> build/lib
installing to /tmp/tmpxdplyp_x
running install
running install_lib
copying build/lib/toxic_tweets_transform.py -> /tmp/tmpxdplyp_x
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmpxdplyp_x/tfx_user_code_Transform-0.0+50a1476fa472d79c2139eb1910363e8df69ad53c0c2584ee4ae21000c44b6f80-py3.10.egg-info
running install_scripts
creating /tmp/tmpxdplyp_x/tfx_user_code_Transf

/home/zero/miniconda3/envs/mlops-tfx/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


Processing ./ramaastra-pipeline/toxic-tweets-pipeline/_wheels/tfx_user_code_Transform-0.0+50a1476fa472d79c2139eb1910363e8df69ad53c0c2584ee4ae21000c44b6f80-py3-none-any.whl
Processing ./ramaastra-pipeline/toxic-tweets-pipeline/_wheels/tfx_user_code_Transform-0.0+50a1476fa472d79c2139eb1910363e8df69ad53c0c2584ee4ae21000c44b6f80-py3-none-any.whl
Processing ./ramaastra-pipeline/toxic-tweets-pipeline/_wheels/tfx_user_code_Transform-0.0+50a1476fa472d79c2139eb1910363e8df69ad53c0c2584ee4ae21000c44b6f80-py3-none-any.whl
INFO:tensorflow:Assets written to: ramaastra-pipeline/toxic-tweets-pipeline/Transform/transform_graph/5/.temp_path/tftransform_tmp/2bcb27ea03184286bb88dc8f9df0bbf3/assets


INFO:tensorflow:Assets written to: ramaastra-pipeline/toxic-tweets-pipeline/Transform/transform_graph/5/.temp_path/tftransform_tmp/2bcb27ea03184286bb88dc8f9df0bbf3/assets


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

#### Viewing Transformed Data


In [19]:
pp = PrettyPrinter()

train_uri = os.path.join(
    transform.outputs["transformed_examples"].get()[0].uri, "Split-train"
)

tfrecord_train_filenames = [
    os.path.join(train_uri, name) for name in os.listdir(train_uri)
]

dataset = tf.data.TFRecordDataset(tfrecord_train_filenames, compression_type="GZIP")

for tfrecord in dataset.take(5):
    serialized_example = tfrecord.numpy()
    example = tf.train.Example()
    example.ParseFromString(serialized_example)
    pp.pprint(example)

features {
  feature {
    key: "is_toxic_xf"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "tweet_xf"
    value {
      bytes_list {
        value: "three little birds"
      }
    }
  }
}

features {
  feature {
    key: "is_toxic_xf"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "tweet_xf"
    value {
      bytes_list {
        value: "i like this positive never stress to things we can t control"
      }
    }
  }
}

features {
  feature {
    key: "is_toxic_xf"
    value {
      int64_list {
        value: 1
      }
    }
  }
  feature {
    key: "tweet_xf"
    value {
      bytes_list {
        value: "go ahead and mock me one day i ll sell enough of this make up to drive the pink mary kay cadillac and we ll see who s gay"
      }
    }
  }
}

features {
  feature {
    key: "is_toxic_xf"
    value {
      int64_list {
        value: 1
      }
    }
  }
  feature {
    key: "tweet_xf"
    value {
   

### Model Hyperparameter Tuning


#### Creating the Tuner Module


In [20]:
TUNER_MODULE_FILE = "toxic_tweets_tuner.py"

In [24]:
%%writefile {TUNER_MODULE_FILE}

from typing import NamedTuple, Dict, Text, Any
from keras_tuner.engine import base_tuner
from tensorflow.keras import layers
import tensorflow as tf
import tensorflow_transform as tft
import keras_tuner as kt

LABEL_KEY = "is_toxic"
FEATURE_KEY = "tweet"


def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"


def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type="GZIP")


def input_fn(
    file_pattern, tf_transform_output, num_epochs, batch_size=64
) -> tf.data.Dataset:
    """Get post_tranform feature & create batches of data"""

    # Get post_transform feature spec
    transform_feature_spec = tf_transform_output.transformed_feature_spec().copy()

    # Create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key=transformed_name(LABEL_KEY),
    )

    return dataset


# Vocabulary size and number of words in a sequence
VOCAB_SIZE = 10000
SEQUENCE_LENGTH = 50

vectorize_layer = layers.TextVectorization(
    standardize="lower_and_strip_punctuation",
    max_tokens=VOCAB_SIZE,
    output_mode="int",
    output_sequence_length=SEQUENCE_LENGTH,
)


def model_builder(hp):
    """Builds a keras model to be tuned"""
    embedding_dim = hp.Int("embedding_dim", min_value=16, max_value=64, step=16)
    lstm_units = hp.Int("lstm_units", min_value=32, max_value=64, step=16)
    dense_units = hp.Int("dense_units", min_value=16, max_value=64, step=16)
    dropout_rate = hp.Float("dropout_rate", min_value=0.2, max_value=0.4, step=0.1)
    learning_rate = hp.Choice("learning_rate", values=[1e-2, 1e-3])

    inputs = tf.keras.Input(
        shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string
    )
    reshaped_narrative = tf.reshape(inputs, [-1])
    x = vectorize_layer(reshaped_narrative)
    x = layers.Embedding(VOCAB_SIZE, embedding_dim)(x)
    x = layers.Bidirectional(layers.LSTM(lstm_units))(x)
    x = layers.Dense(dense_units, activation="relu")(x)
    x = layers.Dropout(dropout_rate)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    model.compile(
        loss=tf.keras.losses.BinaryCrossentropy(),
        optimizer=tf.keras.optimizers.Adam(learning_rate),
        metrics=[tf.keras.metrics.BinaryAccuracy()],
    )

    return model


def tuner_fn(fn_args):
    """Build the tuner using the KerasTuner API.
    Args:
      fn_args: Holds args used to tune models as name/value pairs.

    Returns:
      A namedtuple contains the following:
        - tuner: A BaseTuner that will be used for tuning.
        - fit_kwargs: Args to pass to tuner"s run_trial function for fitting the
                      model , e.g., the training and validation dataset. Required
                      args depend on the above tuner"s implementation.
    """
    # Load the transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    train_set = input_fn(fn_args.train_files[0], tf_transform_output, num_epochs=10)
    val_set = input_fn(fn_args.eval_files[0], tf_transform_output, num_epochs=10)

    vectorize_layer.adapt(
        [
            j[0].numpy()[0]
            for j in [i[0][transformed_name(FEATURE_KEY)] for i in list(train_set)]
        ]
    )

    # Define the tuner
    tuner = kt.Hyperband(
        lambda hp: model_builder(hp),
        objective="val_binary_accuracy",
        max_epochs=5,
        factor=3,
        directory=fn_args.working_dir,
        project_name="toxic_tweets_tuner",
    )

    # Define early stopping callback
    stop_early = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=1)

    TunerFnResult = NamedTuple(
        "TunerFnResult",
        [("tuner", base_tuner.BaseTuner), ("fit_kwargs", Dict[Text, Any])],
    )

    return TunerFnResult(
        tuner=tuner,
        fit_kwargs={
            "callbacks": [stop_early],
            "x": train_set,
            "validation_data": val_set,
            "steps_per_epoch": fn_args.train_steps,
            "validation_steps": fn_args.eval_steps,
        },
    )

Writing toxic_tweets_tuner.py


#### Creating the Tuner Component and Running the Tuner


In [25]:
tuner = Tuner(
    module_file=os.path.abspath(TUNER_MODULE_FILE),
    examples=transform.outputs["transformed_examples"],
    transform_graph=transform.outputs["transform_graph"],
    schema=schema_gen.outputs["schema"],
    train_args=trainer_pb2.TrainArgs(splits=["train"]),
    eval_args=trainer_pb2.EvalArgs(splits=["eval"]),
)

interactive_context.run(tuner)

Trial 10 Complete [00h 07m 38s]
val_binary_accuracy: 0.9183816909790039

Best val_binary_accuracy So Far: 0.9227792620658875
Total elapsed time: 01h 19m 12s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Results summary
Results in ramaastra-pipeline/toxic-tweets-pipeline/.temp/7/toxic_tweets_tuner
Showing 10 best trials
Trial summary
Hyperparameters:
embedding_dim: 48
lstm_units: 48
dense_units: 64
dropout_rate: 0.4
learning_rate: 0.01
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 0.9227792620658875
Trial summary
Hyperparameters:
embedding_dim: 64
lstm_units: 64
dense_units: 48
dropout_rate: 0.2
learning_rate: 0.01
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 0.9227792620658875
Trial summary
Hyperparameters:
embedding_dim: 48
lstm_units: 32
dense_units: 16
dropout_rate: 0.30000000000000004
learning_rate: 0.01
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 0.9203166365623474
Trial summary
Hyperparameters:
embedding_dim: 48
lstm_units: 48
dense_units: 64
dropout_rate: 0.4
learning_rate: 0.01
tuner/epochs: 5
tuner/initial_epoch: 2
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0000
Score: 0.92

ExecutionResult(
    component_id: Tuner
    execution_id: 7
    outputs:
        best_hyperparameters: OutputChannel(artifact_type=HyperParameters, producer_component_id=Tuner, output_key=best_hyperparameters, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        tuner_results: OutputChannel(artifact_type=TunerResults, producer_component_id=Tuner, output_key=tuner_results, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

### Model Training


#### Creating the Trainer Module


In [26]:
TRAINER_MODULE_FILE = "toxic_tweets_trainer.py"

In [27]:
%%writefile {TRAINER_MODULE_FILE}

import os
import tensorflow as tf
import tensorflow_transform as tft
from tensorflow.keras import layers
import tensorflow_hub as hub

LABEL_KEY = "is_toxic"
FEATURE_KEY = "tweet"


def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"


def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type="GZIP")


def input_fn(
    file_pattern, tf_transform_output, num_epochs, batch_size=64
) -> tf.data.Dataset:
    """Get post_tranform feature & create batches of data"""

    # Get post_transform feature spec
    transform_feature_spec = tf_transform_output.transformed_feature_spec().copy()

    # Create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key=transformed_name(LABEL_KEY),
    )
    return dataset


# Vocabulary size and number of words in a sequence
VOCAB_SIZE = 10000
SEQUENCE_LENGTH = 50

vectorize_layer = layers.TextVectorization(
    standardize="lower_and_strip_punctuation",
    max_tokens=VOCAB_SIZE,
    output_mode="int",
    output_sequence_length=SEQUENCE_LENGTH,
)


def model_builder(hp):
    """Build machine learning model"""
    inputs = tf.keras.Input(
        shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string
    )
    reshaped_narrative = tf.reshape(inputs, [-1])
    x = vectorize_layer(reshaped_narrative)
    x = layers.Embedding(VOCAB_SIZE, hp["embedding_dim"])(x)
    x = layers.Bidirectional(layers.LSTM(hp["lstm_units"]))(x)
    x = layers.Dense(hp["dense_units"], activation="relu")(x)
    x = layers.Dropout(hp["dropout_rate"])(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    model.compile(
        loss=tf.keras.losses.BinaryCrossentropy(),
        optimizer=tf.keras.optimizers.Adam(hp["learning_rate"]),
        metrics=[tf.keras.metrics.BinaryAccuracy()],
    )

    model.summary()

    return model


def _get_serve_tf_examples_fn(model, tf_transform_output):
    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(LABEL_KEY)
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        transformed_features = model.tft_layer(parsed_features)

        # Get predictions using the transformed features
        return model(transformed_features)

    return serve_tf_examples_fn


def run_fn(fn_args):
    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), "logs")
    hp = fn_args.hyperparameters["values"]

    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=log_dir, update_freq="batch"
    )

    stop_early = tf.keras.callbacks.EarlyStopping(
        monitor="val_binary_accuracy", mode="max", verbose=1, patience=5
    )
    model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
        fn_args.serving_model_dir,
        monitor="val_binary_accuracy",
        mode="max",
        verbose=1,
        save_best_only=True,
    )

    # Load the transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    # Create batches of data
    train_set = input_fn(fn_args.train_files, tf_transform_output, hp["tuner/epochs"])
    val_set = input_fn(fn_args.eval_files, tf_transform_output, hp["tuner/epochs"])
    vectorize_layer.adapt(
        [
            j[0].numpy()[0]
            for j in [i[0][transformed_name(FEATURE_KEY)] for i in list(train_set)]
        ]
    )

    # Build the model
    model = model_builder(hp)

    # Train the model
    model.fit(
        x=train_set,
        validation_data=val_set,
        callbacks=[tensorboard_callback, stop_early, model_checkpoint],
        steps_per_epoch=fn_args.train_steps,
        validation_steps=fn_args.eval_steps,
        epochs=hp["tuner/epochs"],
    )
    signatures = {
        "serving_default": _get_serve_tf_examples_fn(
            model, tf_transform_output
        ).get_concrete_function(
            tf.TensorSpec(shape=[None], dtype=tf.string, name="examples")
        )
    }
    model.save(fn_args.serving_model_dir, save_format="tf", signatures=signatures)

Writing toxic_tweets_trainer.py


#### Training the Model


In [28]:
trainer = Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE),
    examples=transform.outputs["transformed_examples"],
    transform_graph=transform.outputs["transform_graph"],
    schema=schema_gen.outputs["schema"],
    hyperparameters=tuner.outputs["best_hyperparameters"],
    train_args=trainer_pb2.TrainArgs(splits=["train"]),
    eval_args=trainer_pb2.EvalArgs(splits=["eval"]),
)

interactive_context.run(trainer)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying toxic_tweets_transform.py -> build/lib
copying toxic_tweets_trainer.py -> build/lib
copying toxic_tweets_tuner.py -> build/lib
installing to /tmp/tmpambh558r
running install
running install_lib
copying build/lib/toxic_tweets_transform.py -> /tmp/tmpambh558r
copying build/lib/toxic_tweets_trainer.py -> /tmp/tmpambh558r
copying build/lib/toxic_tweets_tuner.py -> /tmp/tmpambh558r
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /t

/home/zero/miniconda3/envs/mlops-tfx/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


Processing ./ramaastra-pipeline/toxic-tweets-pipeline/_wheels/tfx_user_code_Trainer-0.0+3cad926db4fb864440d1d01875dfaab1a47993c010208af84151a3c15015b2d7-py3-none-any.whl
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tweet_xf (InputLayer)       [(None, 1)]               0         
                                                                 
 tf.reshape_1 (TFOpLambda)   (None,)                   0         
                                                                 
 text_vectorization (TextVe  (None, 50)                0         
 ctorization)                                                    
                                                                 
 embedding_1 (Embedding)     (None, 50, 48)            480000    
                                                                 
 bidirectional_1 (Bidirecti  (None, 96)                37248     
 onal)               

INFO:tensorflow:Assets written to: ramaastra-pipeline/toxic-tweets-pipeline/Trainer/model/8/Format-Serving/assets


1418/1418 [==============================] - 67s 45ms/step - loss: 0.2033 - binary_accuracy: 0.9229 - val_loss: 0.2215 - val_binary_accuracy: 0.9214
Epoch 2/2
1417/1418 [============================>.] - ETA: 0s - loss: 0.1452 - binary_accuracy: 0.9436
Epoch 2: val_binary_accuracy did not improve from 0.92137
1418/1418 [==============================] - 59s 42ms/step - loss: 0.1452 - binary_accuracy: 0.9436 - val_loss: 0.2514 - val_binary_accuracy: 0.9207
INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: ramaastra-pipeline/toxic-tweets-pipeline/Trainer/model/8/Format-Serving/assets


INFO:tensorflow:Assets written to: ramaastra-pipeline/toxic-tweets-pipeline/Trainer/model/8/Format-Serving/assets


ExecutionResult(
    component_id: Trainer
    execution_id: 8
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

### Model Analysis and Deployment


#### Creating the Resolver Component


In [29]:
model_resolver = Resolver(
    strategy_class=LatestBlessedModelStrategy,
    model=Channel(type=Model),
    model_blessing=Channel(type=ModelBlessing),
).with_id("Latest_blessed_model_resolver")

interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 9
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

#### Creating EvalConfig


In [30]:
eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key="is_toxic")],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[
        tfma.MetricsSpec(
            metrics=[
                tfma.MetricConfig(class_name="ExampleCount"),
                tfma.MetricConfig(class_name="AUC"),
                tfma.MetricConfig(class_name="FalsePositives"),
                tfma.MetricConfig(class_name="TruePositives"),
                tfma.MetricConfig(class_name="FalseNegatives"),
                tfma.MetricConfig(class_name="TrueNegatives"),
                tfma.MetricConfig(
                    class_name="BinaryAccuracy",
                    threshold=tfma.MetricThreshold(
                        value_threshold=tfma.GenericValueThreshold(
                            lower_bound={"value": 0.5}
                        ),
                        change_threshold=tfma.GenericChangeThreshold(
                            direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                            absolute={"value": 0.0001},
                        ),
                    ),
                ),
            ]
        )
    ],
)

#### Creating the Evaluator Component


In [31]:
evaluator = Evaluator(
    examples=example_gen.outputs["examples"],
    model=trainer.outputs["model"],
    baseline_model=model_resolver.outputs["model"],
    eval_config=eval_config,
)

interactive_context.run(evaluator)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 10
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

#### Viewing the Evaluation Results


In [50]:
eval_result = evaluator.outputs["evaluation"].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)

SlicingMetricsViewer(config={'weightedExamplesColumn': 'example_count'}, data=[{'slice': 'Overall', 'metrics':…

In [78]:
pp.pprint(tfma_result.slicing_metrics[0][1])

{'': {'': {'auc': {'doubleValue': 0.9644376271132888},
           'binary_accuracy': {'doubleValue': 0.9206684256816183},
           'example_count': {'doubleValue': 11370.0},
           'false_negatives': {'doubleValue': 627.0},
           'false_positives': {'doubleValue': 275.0},
           'loss': {'doubleValue': 0.25139012932777405},
           'true_negatives': {'doubleValue': 6266.0},
           'true_positives': {'doubleValue': 4202.0}}}}


In [51]:
tfma.addons.fairness.view.widget_view.render_fairness_indicator(tfma_result)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'binary_accu…

In [34]:
eval_result = evaluator.outputs["evaluation"].get()[0].uri
print(tfma.load_validation_result(eval_result))

validation_ok: true
validation_details {
  slicing_details {
    slicing_spec {
    }
    num_matching_slices: 1
  }
}



## Model Deployment


In [36]:
from tfx.components import Pusher
from tfx.proto import pusher_pb2

### Creating the Pusher Component


In [37]:
pusher = Pusher(
    model=trainer.outputs["model"],
    model_blessing=evaluator.outputs["blessing"],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory="serving_model_dir/toxic-tweets-detection-model"
        )
    ),
)

interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 11
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))